### Objective

Using spotipy API to retrieve songs from various playlists.  
**Third step is retrieving the song id's from the playlists.**

In order to not send too many API request of same nature, I am saving the intermediate result. 

*Output: 4_Song_Ids.csv*, columns 'category_id', 'category_name', 'playlist_id', 'playlist_name', 'song_id', 'song_artist', song_title

In [1]:
import pandas as pd
from spotipy_util import retrieve_sp
import time

> retrieve categories

In [2]:
playlist_df = pd.read_csv('./cache/3_Playlist_Ids.csv')
display(playlist_df.head(1))
display(playlist_df.shape)

,category_id,category_name,playlist_id,playlist_name
0,0JQ5DAqbMKFQ00XGBls6ym,Hip-Hop,37i9dQZF1DX36edUJpD76c,Modus Mio


(230, 4)

In [3]:
playlist_df['category_name'].unique()

array(['Hip-Hop', 'Pop', 'Mood', 'Rock', 'Charts', 'In the car',
       'Dance/Electronic', 'Chill', 'Indie', 'Fresh Finds', 'EQUAL',
       'RADAR', 'Workout', 'Sleep', 'Party', 'At Home', 'Focus',
       'Decades', 'Latin', 'R&B', 'Love', 'Kids & Family', 'Metal',
       'Jazz', 'Trending', 'Classical', 'Folk & Acoustic', 'Country',
       'Disney', 'Soul', 'Gaming', 'GLOW', 'TV & Movies', 'Instrumental',
       'Wellness', 'Punk', 'Ambient', 'Blues', 'Cooking & Dining',
       'Alternative', 'Travel', 'Reggae', 'Caribbean', 'Afro',
       'Songwriters', 'Funk & Disco'], dtype=object)

> connect to api

In [4]:
sp = retrieve_sp()

> retrieve all song id's, artists, titles from each playlist

In [5]:
all_songs = []

for idx, row in playlist_df.iterrows():
    playlist_id = row['playlist_id']
    category_id = row['category_id']
    category_name = row['category_name']
    
    # Get playlist details
    playlist = sp.playlist(playlist_id)
    playlist_name = playlist['name']
    
    # Get tracks for the current playlist
    tracks = sp.playlist_tracks(playlist_id, fields='items.track.id,items.track.artists,items.track.name,total')

    # Check if tracks is not None and 'items' in tracks
    if tracks is not None and 'items' in tracks:
        # Iterate over each track and append its data to the list
        for item in tracks['items']:
            # Check if 'track' key exists in the item and it's not None
            if 'track' in item and item['track'] is not None:
                # Check if 'id' key exists in the track
                if 'id' in item['track']:
                    song_id = item['track']['id']
                    song_title = item['track']['name']
                    
                    # Join artists' names if they exist
                    if 'artists' in item['track'] and item['track']['artists']:
                        song_artist = ', '.join([artist['name'] for artist in item['track']['artists'] if 'name' in artist])
                    else:
                        song_artist = None  # or any other appropriate placeholder
                    
                    all_songs.append({'category_id': category_id, 'category_name': category_name, 'playlist_id': playlist_id, 'playlist_name': playlist_name, 'song_id': song_id, 'song_artist': song_artist, 'song_title': song_title})

    # Introduce a delay between requests, increasing with each iteration
    time.sleep(2)

ReadTimeout: HTTPSConnectionPool(host='api.spotify.com', port=443): Read timed out. (read timeout=5)

> check, remove duplicates

In [6]:
songs_df = pd.DataFrame(all_songs)
songs_df.shape

(15629, 7)

In [7]:
songs_df.drop_duplicates(inplace=True) 
songs_df.shape   

(15629, 7)

In [12]:
songs_df

,category_id,category_name,playlist_id,playlist_name,song_id,song_artist,song_title
0,0JQ5DAqbMKFQ00XGBls6ym,Hip-Hop,37i9dQZF1DX36edUJpD76c,Modus Mio,6df3If2RzcZ0CP79sVEuSe,"Olexesh, Bonez MC",Cabrio
1,0JQ5DAqbMKFQ00XGBls6ym,Hip-Hop,37i9dQZF1DX36edUJpD76c,Modus Mio,3pPMgoouYtezLiBTEeL9be,"FOURTY, Bausa",VEMPA
2,0JQ5DAqbMKFQ00XGBls6ym,Hip-Hop,37i9dQZF1DX36edUJpD76c,Modus Mio,6aCAQmJMBHve3Tfelte1HR,"Dardan, Azet",Pa Mu
3,0JQ5DAqbMKFQ00XGBls6ym,Hip-Hop,37i9dQZF1DX36edUJpD76c,Modus Mio,16f4ZN6XOfrIO8fJkm28HM,"AYLIVA, Apache 207",Wunder
4,0JQ5DAqbMKFQ00XGBls6ym,Hip-Hop,37i9dQZF1DX36edUJpD76c,Modus Mio,5g6kQitL4OSo9tWkWDGkNT,"Celo & Abdi, Summer Cem, Nimo",So So Def (feat. Summer Cem & Nimo)
...,...,...,...,...,...,...,...
15624,0JQ5DAqbMKFRY5ok2pxXJ0,Cooking & Dining,37i9dQZF1DX5Vpt0fhumU1,Spanish Tapas Bar,2SbLBxUtlSzz8nIYFQstwq,Govi,Samba Delight
15625,0JQ5DAqbMKFRY5ok2pxXJ0,Cooking & Dining,37i9dQZF1DX5Vpt0fhumU1,Spanish Tapas Bar,6ExQLKsX71y57iO7iA8Ggd,Johannes Linstead,Andalucia
15626,0JQ5DAqbMKFRY5ok2pxXJ0,Cooking & Dining,37i9dQZF1DX5Vpt0fhumU1,Spanish Tapas Bar,7A1prltPZsLqZ1iTROIkhz,"Traditional, Alhama, Los",El Lerele
15627,0JQ5DAqbMKFRY5ok2pxXJ0,Cooking & Dining,37i9dQZF1DX5Vpt0fhumU1,Spanish Tapas Bar,0aiqrNJkcz1xDGUD8HLrRz,Govi,Bumblebeat


> save to csv, convert the list of tuples to a DataFrame

In [11]:
songs_df.to_csv('./cache/4_Song_Ids.csv', index=False)